In [2]:
from frykit import DATA_DIRPATH
from frykit._shp import convert_gcj_to_wgs, BinaryConverter

In [ ]:
shp_dirpath = DATA_DIRPATH / 'shp'
gcj_filepath = shp_dirpath / 'province.shp'
wgs_filepath = shp_dirpath / 'new_province.shp'
convert_gcj_to_wgs(gcj_filepath, wgs_filepath)

with BinaryConverter(shp_dirpath / 'province.bin') as converter:
    converter.convert(wgs_filepath)